In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ammi-bootcamp-kaggle-competition/Sample_Submission.csv
/kaggle/input/ammi-bootcamp-kaggle-competition/train.csv
/kaggle/input/ammi-bootcamp-kaggle-competition/test.csv


In [2]:
import cupy as np
import cudf as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [3]:
train_gdf=pd.read_csv('/kaggle/input/ammi-bootcamp-kaggle-competition/train.csv')
test_gdf=pd.read_csv('/kaggle/input/ammi-bootcamp-kaggle-competition/test.csv')

In [4]:
train_gdf['points'] = train_gdf['points'].astype(int)
test_gdf['points'] = test_gdf['points'].astype(int)

In [5]:
train_gdf.set_index('id',inplace=True)
test_gdf.set_index('id',inplace=True)

In [6]:
train_gdf

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
id,,,,,,,,,,,,,
32027,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88,20.0,Alentejano,<NA>,<NA>,<NA>,<NA>,<NA>,PORTUGUESE RED,J. Portugal Ramos
71079,France,"A solid, chunky wine, with a structure that is...",<NA>,88,28.0,Bordeaux,Lalande de Pomerol,<NA>,<NA>,<NA>,<NA>,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier
32440,France,"This is powerful and concentrated, with the hi...",<NA>,94,130.0,Bordeaux,Saint-Émilion,<NA>,<NA>,<NA>,<NA>,BORDEAUX-STYLE RED BLEND,Château Figeac
124405,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89,34.0,California,Santa Barbara County,Central Coast,<NA>,<NA>,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs
33649,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89,24.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51364,US,"Named for the clone so beloved by winemakers, ...",Old Wente,93,55.0,California,Russian River Valley,Sonoma,Virginie Boone,@vboone,Lynmar 2014 Old Wente Chardonnay (Russian Rive...,CHARDONNAY,Lynmar
87223,Portugal,"This is a fresh, fruity and juicy wine. Shot t...",Bojador Branco,85,14.0,Alentejano,<NA>,<NA>,Roger Voss,@vossroger,Espaço Rural 2014 Bojador Branco White (Alente...,PORTUGUESE WHITE,Espaço Rural
142217,Spain,"Clean and toasty, with mineral and the slighte...",Cordon Negro Brut,85,10.0,Catalonia,Cava,<NA>,<NA>,<NA>,<NA>,SPARKLING BLEND,Freixenet


In [7]:
train_gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 175000 entries, 32027 to 91621
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   country                174953 non-null  object
 1   description            175000 non-null  object
 2   designation            122734 non-null  object
 3   points                 175000 non-null  int64
 4   price                  175000 non-null  float64
 5   province               174953 non-null  object
 6   region_1               146466 non-null  object
 7   region_2               75394 non-null   object
 8   taster_name            65509 non-null   object
 9   taster_twitter_handle  62190 non-null   object
 10  title                  82189 non-null   object
 11  variety                174999 non-null  object
 12  winery                 175000 non-null  object
dtypes: float64(1), int64(1), object(11)
memory usage: 68.8+ MB


In [8]:
test_gdf

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
id,,,,,,,,,,,,,,
0,41855,US,"Sweet new oak stands out, giving this Cabernet...",3D,95,<NA>,California,St. Helena,Napa,<NA>,<NA>,Salvestrin 2007 3D Cabernet Sauvignon (St. Hel...,CABERNET SAUVIGNON,Salvestrin
1,10328,France,Charming raspberry aromas intertwine with scen...,Heluicum,90,<NA>,Rhône Valley,Collines Rhôdaniennes,<NA>,<NA>,<NA>,<NA>,SYRAH,Les Vins de Vienne
2,60094,US,"Ripe blackberry, leather and soy show on the n...",Estate,88,<NA>,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Brave & Maiden 2013 Estate Cabernet Franc (San...,CABERNET FRANC,Brave & Maiden
3,48333,US,White flowers and wild anise give this a crisp...,<NA>,89,<NA>,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Les Voleurs 2013 Chardonnay (Sonoma Coast),CHARDONNAY,Les Voleurs
4,14498,US,Pinot Gris has been making inroads in Washingt...,<NA>,88,<NA>,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Chateau Ste. Michelle 2015 Pinot Gris (Columbi...,PINOT GRIS,Chateau Ste. Michelle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83205,48152,US,"A simple, direct wine, pretty full-bodied for ...",Le Pique-Nique,81,<NA>,California,Sonoma Valley,Sonoma,<NA>,<NA>,<NA>,ROSé,Wellington
83206,103366,US,"This dry, spicy wine shows the rustic, rugged ...",Rebecca's Vineyard,87,<NA>,California,Dry Creek Valley,Sonoma,<NA>,<NA>,<NA>,CABERNET SAUVIGNON,Forth
83207,145186,Italy,"Attractive apricot, peach and honey aromas vie...",Aurente,89,<NA>,Central Italy,Umbria,<NA>,<NA>,<NA>,<NA>,CHARDONNAY,Lungarotti


## We need to handle null values and preprocess data before model building

In [9]:
train_gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 175000 entries, 32027 to 91621
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   country                174953 non-null  object
 1   description            175000 non-null  object
 2   designation            122734 non-null  object
 3   points                 175000 non-null  int64
 4   price                  175000 non-null  float64
 5   province               174953 non-null  object
 6   region_1               146466 non-null  object
 7   region_2               75394 non-null   object
 8   taster_name            65509 non-null   object
 9   taster_twitter_handle  62190 non-null   object
 10  title                  82189 non-null   object
 11  variety                174999 non-null  object
 12  winery                 175000 non-null  object
dtypes: float64(1), int64(1), object(11)
memory usage: 68.8+ MB


### Twitter handle, taster_name, id are irrelevant for model training so we drop these from test and train

In [10]:
train_gdf=train_gdf.drop(['taster_name','taster_twitter_handle','title'],axis=1)

test_gdf=test_gdf.drop(['taster_name','taster_twitter_handle','title','price','index'],axis=1)

In [11]:
train_gdf['points'].describe()

count    175000.000000
mean         87.581606
std           3.195285
min          79.000000
25%          85.000000
50%          87.000000
75%          90.000000
max         100.000000
Name: points, dtype: float64

In [12]:
from collections import Counter
Counter(train_gdf['points'].to_pandas())

Counter({88: 18585,
         94: 3183,
         89: 17616,
         83: 8353,
         85: 15840,
         81: 2676,
         92: 9466,
         87: 22729,
         82: 4911,
         84: 12338,
         86: 20665,
         90: 16374,
         91: 12508,
         95: 1332,
         80: 1231,
         93: 5866,
         96: 521,
         79: 423,
         97: 243,
         98: 86,
         99: 37,
         100: 17})

Missing values

In [13]:
print('Missing values in train')
total = train_gdf.isnull().sum().sort_values(ascending=False)
percent = (train_gdf.isnull().sum()/train_gdf.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1)
missing_data.head(30)

Missing values in train


,0,1
region_2,99606,0.569177
designation,52266,0.298663
region_1,28534,0.163051
country,47,0.000269
province,47,0.000269
variety,1,0.000006
description,0,0.000000
points,0,0.000000
price,0,0.000000
winery,0,0.000000


In [14]:
print('Missing values in test')
total = test_gdf.isnull().sum().sort_values(ascending=False)
percent = (test_gdf.isnull().sum()/test_gdf.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1)
missing_data.head(30)

Missing values in test


,0,1
region_2,47608,0.572143
designation,24824,0.298330
region_1,13883,0.166843
country,17,0.000204
province,17,0.000204
description,0,0.000000
points,0,0.000000
variety,0,0.000000
winery,0,0.000000


In [15]:
print(train_gdf['region_2'].describe())

count             75394
unique               19
top       Central Coast
freq              16174
Name: region_2, dtype: object


Region 2 has lot of missing values so we drop it

In [16]:
train_gdf.drop('region_2',axis=1,inplace=True)
test_gdf.drop('region_2',axis=1,inplace=True)

## Filling mode wont make sense as province and country and designation are interrelated so we fill "missing"

In [17]:
for i in train_gdf.columns:
    if train_gdf[i].isnull().sum() >= 1:
        train_gdf[i].fillna('missing',inplace = True)

print('The missing values in the test data are as shown below:')
print('+*+'*20)
train_gdf.isnull().sum().sort_values(ascending=False) 

The missing values in the test data are as shown below:
+*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*+


country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
variety        0
winery         0
dtype: uint64

In [18]:
for i in test_gdf.columns:
    if test_gdf[i].isnull().sum() >= 1:
        test_gdf[i].fillna('missing',inplace = True)

print('The missing values in the test data are as shown below:')
print('+*+'*20)
test_gdf.isnull().sum().sort_values(ascending=False) 

The missing values in the test data are as shown below:
+*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*++*+


country        0
description    0
designation    0
points         0
province       0
region_1       0
variety        0
winery         0
dtype: uint64

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
sia=SentimentIntensityAnalyzer()

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [20]:
def punc_clean(texL): #Removing punctuations
    import string as st
    newl=[]
    for text in texL.to_pandas():
        a=[w for w in text if w not in st.punctuation]
        newl.append(''.join(a))
    return newl
train_gdf['description']=punc_clean(train_gdf['description'])
test_gdf['description']=punc_clean(test_gdf['description'])
train_gdf.head(2)

,country,description,designation,points,price,province,region_1,variety,winery
id,,,,,,,,,
32027,Portugal,This is a fine rich balanced wine It has ripe ...,Vila Santa Reserva,88,20.0,Alentejano,missing,PORTUGUESE RED,J. Portugal Ramos
71079,France,A solid chunky wine with a structure that is f...,missing,88,28.0,Bordeaux,Lalande de Pomerol,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier


In [21]:
def remove_stopword(textL):
    newL=[]
    for text in textL.to_pandas():
        stopword=nltk.corpus.stopwords.words('english')
        stopword.remove('not')
        a=[w for w in nltk.word_tokenize(text) if w not in stopword]
        newL.append(' '.join(a))
    return newL
train_gdf['description']=remove_stopword(train_gdf['description'])
test_gdf['description']=remove_stopword(test_gdf['description'])
train_gdf.head(2)

,country,description,designation,points,price,province,region_1,variety,winery
id,,,,,,,,,
32027,Portugal,This fine rich balanced wine It ripe fruits su...,Vila Santa Reserva,88,20.0,Alentejano,missing,PORTUGUESE RED,J. Portugal Ramos
71079,France,A solid chunky wine structure full corners edg...,missing,88,28.0,Bordeaux,Lalande de Pomerol,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier


In [22]:
newL=[]

for j in train_gdf['description'].to_pandas():
    cp=sia.polarity_scores(j)['compound']
    
    fin=1
    if cp>=0.05:
        fin=2
    elif cp>-0.05:
        fin=1
    else:
        fin=0
    
    newL.append(fin)
train_gdf['description']=newL

In [23]:
newL=[]
for j in test_gdf['description'].to_pandas():
    cp=sia.polarity_scores(j)['compound']
    
    fin=1
    if cp>=0.05:
        fin=2
    elif cp>-0.05:
        fin=1
    else:
        fin=0
    
    newL.append(fin)
test_gdf['description']=newL

In [24]:
train_gdf['points'] = train_gdf['points'].astype(int)
test_gdf['points'] = test_gdf['points'].astype(int)

In [25]:
test_gdf.head()

,country,description,designation,points,province,region_1,variety,winery
id,,,,,,,,
0,US,2,3D,95,California,St. Helena,CABERNET SAUVIGNON,Salvestrin
1,France,2,Heluicum,90,Rhône Valley,Collines Rhôdaniennes,SYRAH,Les Vins de Vienne
2,US,2,Estate,88,California,Santa Ynez Valley,CABERNET FRANC,Brave & Maiden
3,US,1,missing,89,California,Sonoma Coast,CHARDONNAY,Les Voleurs
4,US,2,missing,88,Washington,Columbia Valley (WA),PINOT GRIS,Chateau Ste. Michelle


In [26]:
train_gdf.head()

,country,description,designation,points,price,province,region_1,variety,winery
id,,,,,,,,,
32027,Portugal,2,Vila Santa Reserva,88,20.0,Alentejano,missing,PORTUGUESE RED,J. Portugal Ramos
71079,France,2,missing,88,28.0,Bordeaux,Lalande de Pomerol,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier
32440,France,2,missing,94,130.0,Bordeaux,Saint-Émilion,BORDEAUX-STYLE RED BLEND,Château Figeac
124405,US,2,Thompson Vineyard,89,34.0,California,Santa Barbara County,PETITE SIRAH,Jaffurs
33649,US,2,McKinley Springs Vineyard,89,24.0,Washington,Horse Heaven Hills,ROSé,Syncline


Encoding categorical

In [27]:
catfeat = ['country', 'designation', 'province','region_1', 'variety', 'winery']
encoder_dict = {}
for i in catfeat:
    t = train_gdf.groupby([i])['price'].mean().sort_values(ascending=True).index
    encoder_dict[i] = {k:i for i,k in enumerate(t.to_pandas(),0)}



In [28]:
for i in catfeat:
    newl=[]
    for j in train_gdf[i].to_pandas():
        newl.append(encoder_dict[i][j])
    train_gdf[i]=newl

In [29]:
train_gdf

,country,description,designation,points,price,province,region_1,variety,winery
id,,,,,,,,,
32027,28,2,11808,88,20.0,226,713,403,5690
71079,42,2,19115,88,28.0,428,901,663,10157
32440,42,2,19115,94,130.0,428,1190,663,16823
124405,39,2,24323,89,34.0,411,828,519,12963
33649,39,2,19138,89,24.0,364,878,214,10277
...,...,...,...,...,...,...,...,...,...
51364,39,2,31770,93,55.0,411,1016,551,15263
87223,28,2,4790,85,14.0,226,713,125,8502
142217,32,2,2256,85,10.0,329,322,487,1960


In [30]:
for i in catfeat:
    newl=[]
    for j in test_gdf[i].to_pandas():
        if j not in encoder_dict[i].keys():
            newl.append(None)
        else:
            newl.append(encoder_dict[i][j])
    test_gdf[i]=newl

In [31]:
test_gdf.fillna(test_gdf.median(),inplace=True)

In [32]:
test_gdf

,country,description,designation,points,province,region_1,variety,winery
id,,,,,,,,
0,39,2,34934,95,411,1187,643,15308
1,42,2,19115,90,433,176,600,14931
2,39,2,22284,88,411,908,559,16002
3,39,1,19115,89,411,1101,551,13099
4,39,2,19115,88,364,758,357,7677
...,...,...,...,...,...,...,...,...
83205,39,2,4572,81,411,1014,214,8076
83206,39,1,24745,87,411,854,643,8814
83207,40,2,27174,89,310,930,551,12868


## Model Training

In [33]:
train_gdf = train_gdf.astype('float32')
test_gdf=test_gdf.astype('float32')

In [34]:
Y = train_gdf['price']
X = train_gdf.drop(['price'],axis=1)

In [35]:
from cuml.preprocessing import StandardScaler
transformer = StandardScaler().fit(X)
xtrain_scaled=transformer.transform(X)

In [36]:
xtrain_scaled

,0,1,2,3,4,5,6,7
0,-1.158221,0.417187,-0.848380,0.130942,-1.358003,-0.239341,-0.744715,-0.876733
1,0.811656,0.417187,-0.008908,0.130942,0.792336,0.401586,1.008269,0.105041
2,0.811656,0.417187,-0.008908,2.008716,0.792336,1.386840,1.008269,1.570120
3,0.389540,0.417187,0.589418,0.443904,0.611367,0.152715,0.037385,0.721754
4,0.389540,0.417187,-0.006266,0.443904,0.111041,0.323175,-2.018999,0.131415
...,...,...,...,...,...,...,...,...
174995,0.389540,0.417187,1.444974,1.695754,0.611367,0.793642,0.253137,1.227257
174996,-1.158221,0.417187,-1.654650,-0.807946,-1.358003,-0.239341,-2.619059,-0.258701
174997,-0.595399,0.417187,-1.945771,-0.807946,-0.261543,-1.572332,-0.178367,-1.696526
174998,0.389540,0.417187,0.355510,0.756867,0.611367,0.793642,0.934104,1.447920


In [37]:
Y

id
32027      20.0
71079      28.0
32440     130.0
124405     34.0
33649      24.0
          ...  
51364      55.0
87223      14.0
142217     10.0
64741      65.0
91621      50.0
Name: price, Length: 175000, dtype: float32

In [38]:
from cuml.model_selection import train_test_split
X_tr, X_te, Y_tr, Y_te = train_test_split(xtrain_scaled, Y, train_size=0.7, random_state=42)

In [39]:
from cuml.ensemble import RandomForestRegressor as curfr
cuml_model = curfr(max_features=1.0, n_bins=128,
                    min_samples_leaf=1,
                    min_samples_split=2,
                    n_estimators=40, accuracy_metric='r2')

In [40]:
cuml_model.fit(X_tr,Y_tr)

RandomForestRegressor()

In [41]:
cuml_score = cuml_model.score(X_tr,Y_tr)
print("MSE score of cuml : ", cuml_score)

MSE score of cuml :  0.8619632720947266


In [42]:
pred=cuml_model.predict(X_te)

In [43]:
pred

0         36.577705
1         50.941013
2         24.684462
3        369.711273
4         13.209142
            ...    
52496     15.851903
52497     38.692596
52498     35.278698
52499     22.329727
52500     32.118912
Length: 52501, dtype: float32

In [44]:
Y_te

id
113301     42.0
71699      30.0
71788      26.0
36528     259.0
45386      13.0
          ...  
45252      16.0
61638      42.0
79238      50.0
13230      20.0
106451     45.0
Name: price, Length: 52501, dtype: float32

In [45]:
import cuml
msq=cuml.metrics.regression.mean_squared_error(Y_te,pred)
r2=cuml.metrics.regression.r2_score(Y_te,pred)
print('msqe is ',msq)
print('r2 is ',r2)

msqe is  234.23979
r2 is  0.8272700309753418


In [46]:
cuml_model = curfr(max_features=1.0, n_bins=128,
                    min_samples_leaf=1,
                    min_samples_split=2,
                    n_estimators=800, accuracy_metric='r2')

In [47]:
cuml_model.fit(xtrain_scaled,Y)

RandomForestRegressor()

In [48]:
xtest_final=test_gdf

In [49]:
xtest_scaled_final=transformer.transform(xtest_final)

In [50]:
preds=cuml_model.predict(xtest_scaled_final)

In [51]:
preds

0        81.606583
1        23.087040
2        55.780151
3        31.113710
4        17.204630
           ...    
83205    14.170022
83206    37.161465
83207    40.075447
83208    25.809692
83209    49.003620
Length: 83210, dtype: float32

In [52]:
tess_gdf=pd.read_csv('/kaggle/input/ammi-bootcamp-kaggle-competition/test.csv')
final_df=pd.DataFrame(tess_gdf['id'],columns=['id'])

In [53]:
final_df['price']=preds

In [54]:
final_df

,id,price
0,0,81.606583
1,1,23.087040
2,2,55.780151
3,3,31.113710
4,4,17.204630
...,...,...
83205,83205,14.170022
83206,83206,37.161465
83207,83207,40.075447
83208,83208,25.809692


In [55]:
filename='./submission.csv'
final_df.to_csv(filename,index=False)